# Exercício 1 - PMR3508
## Classificador KNN para a base *Costa Rican Household Poverty Level Prediction*

### Parte 1: Limpando os dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv('../input/train.csv', 
                         index_col='Id', 
                         na_values='NaN', 
                         engine='python', 
                         sep=r'\s*,\s*')

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

Conferindo campos numéricos e não-numéricos:

In [ ]:
train_data.dtypes

In [ ]:
train_data.shape

Conferindo quais campos possuem valores nulos:

In [ ]:
train_data.isnull().any()

In [ ]:
train_data.dropna().shape

Ou seja, há muitas linhas com valores nulos, de modo que removendo todas elas, o *dataset* fica muito pequeno. No entanto, todos os parâmetros da base são numéricos, de modo que é valido substituir os valores nulos com **zeros**.

In [ ]:
train_data = train_data.fillna(value=0)

Realizando o mesmo procedimento para a base de teste:

In [ ]:
test_data = pd.read_csv('../input/test.csv', 
                         index_col='Id', 
                         na_values='NaN', 
                         engine='python', 
                         sep=r'\s*,\s*')

In [ ]:
test_data.shape

In [ ]:
test_data.dropna().shape

In [ ]:
test_data = test_data.fillna(value=0)

### Parte 2: Explorando os dados

Será feita uma análise de correlação entre os atributos para escolher os mais relevantes:

In [ ]:
train_data.head()

In [ ]:
analise = train_data.corr().loc[:,'Target'].sort_values(ascending=True)
analise

In [ ]:
analise.plot(kind='bar')

### Parte 3: Seleção de Atributos

Selecionando os atributos com o módulo da correlação maior que 0.10:

In [ ]:
analise = train_data.corr().loc[:,'Target'].sort_values(ascending=True).where(lambda x : abs(x) > 0.10).dropna()
analise

In [ ]:
campos = analise.keys().tolist()
campos.remove('Target')

In [ ]:
Xtrain_data = train_data[campos]
Ytrain_data = train_data.Target
Xtest_data = test_data[campos]

### Parte 4: Model Fitting

Primeira tentativa, sem otimização do classificador:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
scores = cross_val_score(knn, Xtrain_data, Ytrain_data, cv=10)

In [ ]:
scores.mean()

Aplicando *GridSearchCV* e *Cross Validation* para otimizar a seleção de hiperparâmetros:

In [ ]:
from sklearn.model_selection import GridSearchCV

k_range = list(range(1,50))
weights = ['uniform', 'distance']
p_range = list(range(1,3))
param = dict(n_neighbors=k_range, p=p_range)

knn = KNeighborsClassifier(n_neighbors=3)
grid = GridSearchCV(knn, param, cv=10, scoring='accuracy', n_jobs = -2)
grid.fit(Xtrain_data, Ytrain_data)
print(grid.best_estimator_)
print(grid.best_score_)

In [ ]:
knn_final = grid.best_estimator_
knn_final.fit(Xtrain_data,Ytrain_data)

In [ ]:
Ytest_data = knn_final.predict(Xtest_data)

In [ ]:
prediction = Ytest_data

In [ ]:
prediction

In [ ]:
Id = test_data.index.values
Id

In [ ]:
s = { 'Id' : Id, 'Target' : prediction.astype(int) }
submission = pd.DataFrame(s)
submission

In [ ]:
submission.to_csv("submission.csv")